In [6]:
from pathlib import Path

In [46]:
ROOT_DIR = Path("../../").resolve()
MODEL_DIR = ROOT_DIR / "src/model"
MODEL_DIR

PosixPath('/media/DataExt4/Software/Others/OpenStudio/src/model')

# Parse Swig .i files to get information about classes

In [99]:
class SwigEntry:
    def __init__(self, name, is_unique, swig_file):
        self.name = name
        self.is_unique = is_unique
        self.swig_file = swig_file

    def __repr__(self):
        s = ""
        if self.is_unique:
            s = ", Unique"
        return f"{self.name}({self.swig_file}{s})"

In [100]:
swig_entries = []
for swig_file in MODEL_DIR.glob("*.i"):
    swig_file_name = swig_file.name
    if swig_file_name in ["Model_Common_Include.i", "Model.i"]:
        continue
    print(swig_file)
    with open(swig_file, "r") as f:
        content = f.read()
    lines = content.splitlines()
    for line in lines:
        line = line.split("//")[0].strip()
        if "UNIQUEMODELOBJECT_TEMPLATES" in line:
            name = line.split("(")[1].split(")")[0].strip()
            swig_entries.append(SwigEntry(name=name, is_unique=True, swig_file=swig_file_name))
        elif "MODELOBJECT_TEMPLATES" in line:
            name = line.split("(")[1].split(")")[0].strip()
            swig_entries.append(SwigEntry(name=name, is_unique=False, swig_file=swig_file_name))

/media/DataExt4/Software/Others/OpenStudio/src/model/ModelAvailabilityManager.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelResources.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelGenerators.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelSimulation.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelGeometry.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelAirflow.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelPlantEquipmentOperationScheme.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelStraightComponent.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelRefrigeration.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelZoneHVAC.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelCore.i
/media/DataExt4/Software/Others/OpenStudio/src/model/ModelHVAC.i


In [101]:
swig_entries.sort(key=lambda x: x.name)
swig_entries

[AdditionalProperties(ModelCore.i),
 AirConditionerVariableRefrigerantFlow(ModelStraightComponent.i),
 AirGap(ModelResources.i),
 AirLoopHVAC(ModelHVAC.i),
 AirLoopHVACDedicatedOutdoorAirSystem(ModelHVAC.i),
 AirLoopHVACOutdoorAirSystem(ModelHVAC.i),
 AirLoopHVACReturnPlenum(ModelHVAC.i),
 AirLoopHVACSupplyPlenum(ModelHVAC.i),
 AirLoopHVACUnitaryHeatCoolVAVChangeoverBypass(ModelStraightComponent.i),
 AirLoopHVACUnitaryHeatPumpAirToAir(ModelStraightComponent.i),
 AirLoopHVACUnitaryHeatPumpAirToAirMultiSpeed(ModelStraightComponent.i),
 AirLoopHVACUnitarySystem(ModelZoneHVAC.i),
 AirLoopHVACZoneMixer(ModelHVAC.i),
 AirLoopHVACZoneSplitter(ModelHVAC.i),
 AirSupplyConstituent(ModelGenerators.i),
 AirTerminalDualDuctConstantVolume(ModelHVAC.i),
 AirTerminalDualDuctVAV(ModelHVAC.i),
 AirTerminalDualDuctVAVOutdoorAir(ModelHVAC.i),
 AirTerminalSingleDuctConstantVolumeCooledBeam(ModelStraightComponent.i),
 AirTerminalSingleDuctConstantVolumeFourPipeBeam(ModelStraightComponent.i),
 AirTerminalSin

# Generate Partial Classes

In [102]:
swig_file = "ModelGeometry.i"
swig_file = "ModelAirflow.i"
swig_file = "ModelSimulation.i"

In [109]:
found_names = [x.name for x in swig_entries if x.is_unique and x.swig_file == swig_file]
if swig_file == "ModelSimulation.i":
    found_names.insert(0, "RunPeriod")
found_names

['RunPeriod',
 'ClimateZones',
 'ConvergenceLimits',
 'EnvironmentalImpactFactors',
 'FoundationKivaSettings',
 'HeatBalanceAlgorithm',
 'InsideSurfaceConvectionAlgorithm',
 'LightingSimulationControl',
 'OutputControlFiles',
 'OutputControlReportingTolerances',
 'OutputDebuggingData',
 'OutputDiagnostics',
 'OutputJSON',
 'OutputTableSummaryReports',
 'OutsideSurfaceConvectionAlgorithm',
 'PerformancePrecisionTradeoffs',
 'RunPeriodControlDaylightSavingTime',
 'ShadowCalculation',
 'SimulationControl',
 'SiteGroundReflectance',
 'SiteGroundTemperatureBuildingSurface',
 'SiteGroundTemperatureDeep',
 'SiteGroundTemperatureFCfactorMethod',
 'SiteGroundTemperatureShallow',
 'SiteWaterMainsTemperature',
 'SizingParameters',
 'Timestep',
 'WeatherFile',
 'YearDescription',
 'ZoneAirContaminantBalance',
 'ZoneAirHeatBalanceAlgorithm',
 'ZoneAirMassFlowConservation',
 'ZoneCapacitanceMultiplierResearchSpecial']

In [110]:
csharp_namespace = "OpenStudio" + swig_file.replace(".i", "")
csharp_namespace

'OpenStudioModelSimulation'

In [111]:
print(
    """#if defined SWIGCSHARP || defined(SWIGJAVA)

  %inline {
    namespace openstudio {
      namespace model {"""
)

for name in found_names:
    getter_name = name[0].lower() + name[1:]
    print(
        f"""
        boost::optional<{name}> {getter_name}(const openstudio::model::Model& model) {{
          return model.{getter_name}();
        }}"""
    )

print(
    """
      }
    }
  } // %inline
  
#endif // defined SWIGCSHARP"""
)

#if defined SWIGCSHARP || defined(SWIGJAVA)

  %inline {
    namespace openstudio {
      namespace model {

        boost::optional<RunPeriod> runPeriod(const openstudio::model::Model& model) {
          return model.runPeriod();
        }

        boost::optional<ClimateZones> climateZones(const openstudio::model::Model& model) {
          return model.climateZones();
        }

        boost::optional<ConvergenceLimits> convergenceLimits(const openstudio::model::Model& model) {
          return model.convergenceLimits();
        }

        boost::optional<EnvironmentalImpactFactors> environmentalImpactFactors(const openstudio::model::Model& model) {
          return model.environmentalImpactFactors();
        }

        boost::optional<FoundationKivaSettings> foundationKivaSettings(const openstudio::model::Model& model) {
          return model.foundationKivaSettings();
        }

        boost::optional<HeatBalanceAlgorithm> heatBalanceAlgorithm(const openstudio::model::Model& mode

In [106]:
print(
    """
#if defined(SWIGCSHARP)
  //%pragma(csharp) imclassimports=%{
  %pragma(csharp) moduleimports=%{

    using System;
    using System.Runtime.InteropServices;

    public partial class Model : Workspace {"""
)

for name in found_names:
    getter_name = name[0].lower() + name[1:]
    print(
        f"""
      public Optional{name} {getter_name}() {{
        return OpenStudio.{csharp_namespace}.{getter_name}(this);
      }}"""
    )

print(
    """
    } // partial class Model
  %} // pragma
#endif // defined(SWIGCSHARP)"""
)


#if defined(SWIGCSHARP)
  //%pragma(csharp) imclassimports=%{
  %pragma(csharp) moduleimports=%{

    using System;
    using System.Runtime.InteropServices;

    public partial class Model : Workspace {

      public OptionalClimateZones climateZones() {
        return OpenStudio.OpenStudioModelSimulation.climateZones(this);
      }

      public OptionalConvergenceLimits convergenceLimits() {
        return OpenStudio.OpenStudioModelSimulation.convergenceLimits(this);
      }

      public OptionalEnvironmentalImpactFactors environmentalImpactFactors() {
        return OpenStudio.OpenStudioModelSimulation.environmentalImpactFactors(this);
      }

      public OptionalFoundationKivaSettings foundationKivaSettings() {
        return OpenStudio.OpenStudioModelSimulation.foundationKivaSettings(this);
      }

      public OptionalHeatBalanceAlgorithm heatBalanceAlgorithm() {
        return OpenStudio.OpenStudioModelSimulation.heatBalanceAlgorithm(this);
      }

      public OptionalIn

In [113]:
print(f"// ignore objects for now, add back in with partial classes in {swig_file}")
for name in found_names:
    getter_name = name[0].lower() + name[1:]
    print(f"  %ignore openstudio::model::Model::{getter_name};")

// ignore objects for now, add back in with partial classes in ModelSimulation.i
  %ignore openstudio::model::Model::runPeriod;
  %ignore openstudio::model::Model::climateZones;
  %ignore openstudio::model::Model::convergenceLimits;
  %ignore openstudio::model::Model::environmentalImpactFactors;
  %ignore openstudio::model::Model::foundationKivaSettings;
  %ignore openstudio::model::Model::heatBalanceAlgorithm;
  %ignore openstudio::model::Model::insideSurfaceConvectionAlgorithm;
  %ignore openstudio::model::Model::lightingSimulationControl;
  %ignore openstudio::model::Model::outputControlFiles;
  %ignore openstudio::model::Model::outputControlReportingTolerances;
  %ignore openstudio::model::Model::outputDebuggingData;
  %ignore openstudio::model::Model::outputDiagnostics;
  %ignore openstudio::model::Model::outputJSON;
  %ignore openstudio::model::Model::outputTableSummaryReports;
  %ignore openstudio::model::Model::outsideSurfaceConvectionAlgorithm;
  %ignore openstudio::model::Mod